In [2]:
import numpy as np

In [3]:
import librosa

/Library/Python/2.7/site-packages/librosa/core/audio.py:37: UserWarning: Could not import scikits.samplerate. Falling back to scipy.signal
  warnings.warn('Could not import scikits.samplerate. '
/Library/Python/2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [4]:
y, sr = librosa.load('data/train/train_0001.mp3')

In [16]:
y

array([  9.31656687e-05,   3.35355056e-04,   6.36704266e-04, ...,
        -1.31993272e-06,  -5.11311691e-06,   1.59234842e-05], dtype=float32)

In [5]:
# collect all training set

from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('data/train') if isfile(join('data/train', f))]

In [57]:
#collect all items in training set
l=[]
for name in onlyfiles:
    y, sr = librosa.load('data/train/'+name)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=100)
    listy=[]
    for item in mfcc:
        listy.append(np.mean(item))
    for item in mfcc:
        listy.append(np.var(item))   
    mfcc_delta = librosa.feature.delta(mfcc)
    for item in mfcc_delta:
        listy.append(np.mean(item))
    for item in mfcc_delta:
        listy.append(np.var(item))
    l.append(listy)    
        

In [192]:
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
listy=[]
for item in mfcc:
    #print item
    listy.append(np.mean(item))
    
mfcc_delta = librosa.feature.delta(mfcc)
mfcc_delta
    
y_harmonic, y_percussive = librosa.effects.hpss(y)    
tempo, beat_frames = librosa.beat.beat_track(y=y,
                                             sr=sr)


chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
                                        sr=sr)

len(chromagram[1])

434

In [7]:
listy

[-144.57782101391584,
 93.298937158925355,
 16.63836005312973,
 25.199496228688687,
 18.879819364064957,
 11.0916612627819,
 10.344652175527685,
 16.726616933764387,
 3.1537225116637617,
 7.9281961848465556,
 0.64580459588731098,
 1.6304821305099793,
 4.0194172165303801,
 3.2539092685030329,
 0.5480830904034858,
 2.5312183080709598,
 -3.9442996010126681,
 -1.0311549427106064,
 3.0994883742763912,
 -5.7691665095473939,
 -0.27175202716427199,
 -0.64183855419898617,
 -4.9722962491182612,
 -3.0944275972503026,
 -1.6473848672267737,
 0.17668846401765065,
 6.2062306299908725,
 -0.17945111986788906,
 -3.8516419516805724,
 -0.63125484935768095,
 -0.53174947939209449,
 3.1341202317398431,
 0.39179317662240615,
 -1.7899237778746853,
 -2.9573805976356287,
 -0.18675575214925594,
 -1.6681914994115636,
 -3.1531950053041609,
 0.52003620988392718,
 -4.3959495643592899,
 -1.5998101561958982,
 -0.95158519528531371,
 -3.7219262849929922,
 -4.3864472575490856,
 -1.98856660246132,
 -3.3184051978131319,
 0.

In [87]:
d=[]
for list1 in l:
    d.append([np.var(list1),np.mean(list1)])


In [8]:
d[1]

NameError: name 'd' is not defined

In [9]:
import pandas as pd

In [10]:
df_labels_train = pd.read_csv('music_train_labels.csv')

In [59]:
y=df_labels_train.genre
y=y.tolist()

In [58]:
X=l

In [172]:
from sklearn import preprocessing
from sklearn.decomposition import PCA


import scipy as sp
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split

class Estimator_prob3(sk.base.BaseEstimator, sk.base.RegressorMixin):
    """
    A shell estimator that combines a transformer and regressor into a single object.
    """
    def __init__(self, transformer, model):
        self.transformer = transformer
        self.model = model
        pass

    def fit(self, X, y):
        X_trans = self.transformer.fit(X)
        self.model.fit(X_trans, y)
        return self
    
    def score(self, X, y):
        X_test = self.transformer.transform(X)
        return self.model.score(X_test, y)

    def predict(self, X):
        X_test = self.transformer.transform(X)
        
        return self.model.predict(X_test)
    
class Transformer_prob3(sk.base.BaseEstimator, sk.base.TransformerMixin):
    """
    Returns the first k columns of a feature array
    """
    def __init__(self):
        pass

    def fit(self, X):
        return X
        
        
    def transform(self, X):   
        return X

In [61]:
from sklearn import preprocessing
from sklearn.decomposition import PCA


import scipy as sp
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split

class Estimator_prob3(sk.base.BaseEstimator, sk.base.RegressorMixin):
    """
    A shell estimator that combines a transformer and regressor into a single object.
    """
    def __init__(self, transformer, model):
        self.transformer = transformer
        self.model = model
        pass

    def fit(self, X, y):
        X_trans = self.transformer.fit(X)
        self.model.fit(X_trans, y)
        
        return self
    
    def score(self, X, y):
        X_test = self.transformer.transform(X)
        return self.model.score(X_test, y)

    def predict(self, X):
        X_test = self.transformer.transform(X)
        X_test=X_test.reshape(1, -1)
        return self.model.predict(X_test)
    
class Transformer_prob3(sk.base.BaseEstimator, sk.base.TransformerMixin):
    """
    Returns the first k columns of a feature array
    """
    def __init__(self):
        self.model= preprocessing.StandardScaler()
        self.pca= PCA(n_components=400)

    def fit(self, X):
        scaler=self.model.fit_transform(X)
        l=[]
        self.model.scale_=self.model.std_.copy()
        for row in scaler:
            l.append(row)
        return self.pca.fit_transform(l)
        
        
    def transform(self, X):
        #X1=self.filter1(X)
        input1=self.model.transform(X)
        l=[]
        for row in input1:
            l.append(row)
        k=self.pca.transform(l)    
        return k

In [62]:
from sklearn import svm
shell = Estimator_prob3(Transformer_prob3(),svm.SVC())

In [63]:
Y=y

In [64]:
shell.fit(X,Y)

/Library/Python/2.7/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function std_ is deprecated; Attribute ``std_`` will be removed in 0.19. Use ``scale_`` instead
  warnings.warn(msg, category=DeprecationWarning)


Estimator_prob3(model=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
        transformer=Transformer_prob3())

In [65]:
from os import listdir
from os.path import isfile, join
onlyfiles1 = [f for f in listdir('data/feature_extraction_test/') if isfile(join('data/feature_extraction_test/', f))]

In [66]:
#collect all items in training set
l1=[]
for name in onlyfiles1:
    y, sr = librosa.load('data/feature_extraction_test/'+name)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=100)
    listy=[]
    for item in mfcc:
        listy.append(np.mean(item))
    for item in mfcc:
        listy.append(np.var(item)) 
    mfcc_delta = librosa.feature.delta(mfcc)
    for item in mfcc_delta:
        listy.append(np.mean(item))
    for item in mfcc_delta:
        listy.append(np.var(item))   
    l1.append(listy)

In [67]:
X_test=l1
len(X_test[1])

400

In [68]:
l=[]
for i in range (0,145):
    l.append(shell.predict(X_test[i]))

/Library/Python/2.7/site-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Library/Python/2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Library/Python/2.7/site-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warni

In [69]:
k=[]
for item in l:
    k.append(item[0])

In [44]:
out={}

In [70]:
out={}
for i in range (0,len(onlyfiles1)):
    out[onlyfiles1[i]]=k[i]

In [48]:
len(k)

145

In [54]:
out

{'fe_test_0001.mp3': 'folkcountry',
 'fe_test_0002.mp3': 'rock',
 'fe_test_0003.mp3': 'jazz',
 'fe_test_0004.mp3': 'rock',
 'fe_test_0005.mp3': 'folkcountry',
 'fe_test_0006.mp3': 'rock',
 'fe_test_0007.mp3': 'folkcountry',
 'fe_test_0008.mp3': 'rock',
 'fe_test_0009.mp3': 'rock',
 'fe_test_0010.mp3': 'raphiphop',
 'fe_test_0011.mp3': 'jazz',
 'fe_test_0012.mp3': 'rock',
 'fe_test_0013.mp3': 'rock',
 'fe_test_0014.mp3': 'rock',
 'fe_test_0015.mp3': 'jazz',
 'fe_test_0016.mp3': 'rock',
 'fe_test_0017.mp3': 'raphiphop',
 'fe_test_0018.mp3': 'raphiphop',
 'fe_test_0019.mp3': 'jazz',
 'fe_test_0020.mp3': 'rock',
 'fe_test_0021.mp3': 'rock',
 'fe_test_0022.mp3': 'rock',
 'fe_test_0023.mp3': 'rock',
 'fe_test_0024.mp3': 'jazz',
 'fe_test_0025.mp3': 'raphiphop',
 'fe_test_0026.mp3': 'rock',
 'fe_test_0027.mp3': 'rock',
 'fe_test_0028.mp3': 'rock',
 'fe_test_0029.mp3': 'raphiphop',
 'fe_test_0030.mp3': 'rock',
 'fe_test_0031.mp3': 'rock',
 'fe_test_0032.mp3': 'jazz',
 'fe_test_0033.mp3': 'jazz

In [56]:
import pickle
with open('dict1.pickle', 'wb') as handle:
  pickle.dump(out, handle)

In [71]:
out

{'fe_test_0001.mp3': 'jazz',
 'fe_test_0002.mp3': 'rock',
 'fe_test_0003.mp3': 'raphiphop',
 'fe_test_0004.mp3': 'rock',
 'fe_test_0005.mp3': 'rock',
 'fe_test_0006.mp3': 'rock',
 'fe_test_0007.mp3': 'rock',
 'fe_test_0008.mp3': 'rock',
 'fe_test_0009.mp3': 'rock',
 'fe_test_0010.mp3': 'raphiphop',
 'fe_test_0011.mp3': 'jazz',
 'fe_test_0012.mp3': 'rock',
 'fe_test_0013.mp3': 'rock',
 'fe_test_0014.mp3': 'rock',
 'fe_test_0015.mp3': 'rock',
 'fe_test_0016.mp3': 'rock',
 'fe_test_0017.mp3': 'raphiphop',
 'fe_test_0018.mp3': 'raphiphop',
 'fe_test_0019.mp3': 'jazz',
 'fe_test_0020.mp3': 'rock',
 'fe_test_0021.mp3': 'jazz',
 'fe_test_0022.mp3': 'rock',
 'fe_test_0023.mp3': 'folkcountry',
 'fe_test_0024.mp3': 'jazz',
 'fe_test_0025.mp3': 'raphiphop',
 'fe_test_0026.mp3': 'folkcountry',
 'fe_test_0027.mp3': 'rock',
 'fe_test_0028.mp3': 'rock',
 'fe_test_0029.mp3': 'rock',
 'fe_test_0030.mp3': 'rock',
 'fe_test_0031.mp3': 'rock',
 'fe_test_0032.mp3': 'jazz',
 'fe_test_0033.mp3': 'jazz',
 'fe

In [72]:
import pickle
with open('dict2.pickle', 'wb') as handle:
  pickle.dump(out, handle)